# Signal Reconstruction Hackathon

### Imports - Do Not Modify


In [1]:
import os
import time

# Force TF to only take the GPU memory it needs, because this can cause
# distributed training to hang for certain GPU/model combinations.
# We must do this prior to importing tensorflow.
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# Enable Tensorflow XLA optimization. This should introduce some overhead
# onto the first step of a train call, but measurably speed up training
# beyond. This chooses different kernels and/or consolidates kernels to
# optimize the graph.
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=2"

import numpy as np
import tensorflow as tf

from tensorflow.python.platform import tf_logging

# Load used functions from autoencoders
from autoencoders.autoencoder import get_train_and_eval_input_functions, \
                                     get_autoencoder
from autoencoders.score_hackathon_models import save_model, \
                                                score_model
import autoencoders.synthetic_data_params as synthetic_data_params

from autoencoders.post_training_utils import predict_on_data

In [83]:
!python -m pip install torch
import torch

^C
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'torch'

### Define parameters
User-defined parameters.

In [104]:


model_dir            = "/domino/datasets/daniel_adams/oct_hackathon_wc/scratch/vae-bi-gru-14/"
learning_rate        = 1e-3
batch_size           = 512
latent_dimension     = "easy" # {"easy", "hard"}
total_training_steps = 1280*5
steps_per_validation = 640
force_training       = True

In [105]:
### Define Embedding Layer

In [106]:
def mfc_embed(layer):
    #input is bsz x 64 x 128 x 1, w == time? 
    
    layer = tf.keras.layers.
    
    return layer

SyntaxError: invalid syntax (<ipython-input-106-15c01bd173aa>, line 4)

### Define Autoencoder Model

#### Encoder

PUT YOUR ENCODER MODEL HERE!!  
This function is passed a single argument, layer, which contains the images for this batch. The shape is (batch_size, 64, 128, 1) i.e. (batch_size, H, W, num_channels). The function must return a Tensor with shape (batch_size, X). This tensor is then passed to a fully connected layer to get the latent features (outside of this function).  

In [107]:
def encoder( layer ):

    # Conv Layers
    #layer = tf.keras.layers.Conv.ConvLSTM2D(filters = 16, kernel_size=3)(layer)
    #layer = tf.keras.layers.Conv2D(filters=64, kernel_size=3)(layer)
    
    
    layerCNN = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3)(layer)
    layerCNN = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(layerCNN)
    
    layerCNN = tf.keras.layers.Flatten()(layerCNN)
    
    layerGRU = tf.keras.layers.GRU(units=256)(tf.squeeze(input=layer, axis=3))
    
    
    layerSiam = tf.keras.layers.concatenate([layerGRU, layerCNN])
    layerEnd = tf.keras.layers.Flatten()(layerSiam)
    layerEnd  = tf.keras.layers.Dense(units=512)(layerSiam)
        
    '''
    layer = tf.squeeze(input=layer, axis=3)
        
    layer = tf.keras.layers.GRU(units=512)(layer)
    
    #layer = tf.keras.layers.MaxPool1D(pool_size=(2))(layer)
    layer = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(tf.expand_dims(layer, axis=2))
    
    layer = tf.keras.layers.Flatten()(layer)
    
    layer = tf.keras.layers.Dense(units=512)(layer)
    '''
    '''
    layer1 = tf.layers.conv2d( layer,
                              filters=16,
                              kernel_size=3,
                              strides=(1, 1),
                              activation=tf.nn.relu,
                              name="encoder_layer_conv_sia1" )
    layer2 = tf.layers.conv2d( layer,
                              filters=32,
                              kernel_size=5,
                              strides=(2, 2),
                              padding=tf.SAME,
                              activation=tf.nn.relu,
                              name="encoder_layer_conv_sia2" )

    
    layer = tf.layers.average_pooling1d(tf.convert_to_tensor([layer1, layer2]),
                              filters=64,
                              kernel_size=3,
                              strides=(1, 1),
                              activation=tf.nn.relu,
                              name="encoder_layer_conv2" )
    
    # MaxPooling Layer
    layer = tf.layers.max_pooling2d( layer,
                                     pool_size=2,
                                     strides=2,
                                     name="encoder_layer_maxpool" )

    # Flatten layer.    
    layer = tf.layers.flatten( layer,
                               name="encoder_layer_flatten" )
    
    # Dense layer.
    layer = tf.layers.dense( layer,
                             units=512,
                             activation=tf.nn.relu,
                             name="encoder_layer_dense1" )
'''
    #layer = tf.keras.layers.ConvLSTM2D(activation=tf.nn.relu, name="encoder_layer_convlstm1", )
    
    return layer

##### Decoder

PUT YOUR DECODER MODEL HERE!!  
This function is passed a single argument, layer, which contains
the latent features for this batch. The shape is (batch_size, N) 
where N is either 64 or 8, depending on the size of the latent layer.
The function must return a Tensor with shape (batch_size, 64, 128, 1),
which is the same shape as the original image.

In [108]:
def decoder( layer ):
    
    layer = tf.keras.layers.Dense(units=512)(layer)
    layer = tf.keras.layers.Reshape(target_shape = (-1, 4, 128))(layer)
    layer = tf.keras.layers.Conv2DTranspose(filters=64,
                                            kernel_size=3, strides=(2, 2))(layer)
    layer = tf.keras.layers.Conv2DTranspose(filters=1,
                                            kernel_size=3, strides=(1, 1))(layer)
    layer = tf.image.resize(layer, (64, 128))
    
    
    '''
    layer = tf.keras.layers.GRU(units=512)(tf.expand_dims(layer, axis=2))
    
    layer = tf.keras.layers.Reshape(target_shape=(4, 128, 1))(layer)

    layer = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2, 2))(layer)
    
    layer = tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=(1, 1))(layer)
    
    layer = tf.image.resize( layer, (64, 128) )

    
    # Dense layer.
    layer = tf.layers.dense( layer,
                             units=512,
                             activation=tf.nn.relu,
                             name="decoder_layer_dense1" )
    
    # Reshape
    layer = tf.reshape( layer, (-1,
                                4,
                                128,
                                1) )
    
    # Conv Transpose layers.
    layer = tf.layers.conv2d_transpose( layer,
                                        filters=64,
                                        kernel_size=3,
                                        strides=(2, 2),
                                        activation=tf.nn.relu,
                                        name="decoder_layer_conv1" )
    
    layer = tf.layers.conv2d_transpose( layer,
                                        filters=1,
                                        kernel_size=3,
                                        strides=(1, 1),
                                        activation=tf.nn.relu,
                                        name="decoder_layer_conv2" )
    
    # Resize to match output shape. Note that this function does
    # bilinear interpolation.
    layer = tf.image.resize( layer, (64, 128) )
    '''
    return layer

##### Model Definition Function 

Modify with caution!

This function defines the full end-to-end autoencoder architecture. Your ENCODER and DECODER function are used here.
  
This can be modified if you want, but we recommend implementing your model through the above ENCODER and DECODER functions.


The user-defined parameters are exposed here as the "params" dictionary. If you decide to touch this function, note that it must return different outputs depending on the model (TRAIN, EVAL, PREDICT). These correspond with the estimator.train(), estimator.eval(), and estimator.predict() function calls.

In [109]:
def full_model_def_func( features,
                         labels,
                         mode=tf.estimator.ModeKeys.TRAIN,
                         params=None ):
    
    if( isinstance( features, dict ) ):
        features = features["images"]
    
        
    # Reshape
    input_layer = tf.reshape( features, (-1,
                                         64,
                                         128,
                                         1) )
    
    # 1. Get the encoder
    encoder_layers = encoder( input_layer )
    
    # 2. Add in the latent layer (depends on 'easy' or 'hard)
    if( params["latent_dimension"] == "easy" ):
        num_latent_dims = 64
    elif( params["latent_dimension"] == "hard" ):
        num_latent_dims = 8
        
    latent_layer = tf.layers.dense(
            encoder_layers,
            units=num_latent_dims,
            activation=None,
            name="latent_layer_units_{}".format( num_latent_dims ) )
    
    # 3. Get the decoder
    decoder_layers = decoder( latent_layer )
    
    # Flatten layer
    # Needs to be (batch_size, 8192) to match the input.
    decoder_layers = tf.layers.flatten( decoder_layers,
                               name="decoder_final_flatten" )
    
    # If predicting, labels are None by default.
    # To calculate rmse, set labels to features.
    if( tf.estimator.ModeKeys.PREDICT == mode ):
        labels = features
        
    # 4. Compute loss and backprop
    squared_error        = tf.pow( labels - decoder_layers, 2 )
    # NOTE: Average across pixels.
    rmse                 = tf.sqrt( tf.reduce_mean( input_tensor=squared_error, axis=1 ) )
    reconstruction_error = tf.reduce_mean( input_tensor=rmse )
    
    if( tf.estimator.ModeKeys.PREDICT == mode ):
        
        # Create prediction outputs.
        predictions = {}
        predictions["latent_features"] = latent_layer

        predictions["recon"]         = decoder_layers
        predictions["squared_error"] = squared_error
        predictions["rmse"]          = rmse
        
        return tf.estimator.EstimatorSpec( mode, predictions=predictions )

    # Loss is reconstruction error.
    loss = reconstruction_error
    
    if( mode == tf.estimator.ModeKeys.EVAL ):
        
        return tf.estimator.EstimatorSpec( mode, loss=loss )
    
    optimizer = tf.compat.v1.train.AdamOptimizer( params["learning_rate"] )
    train_op  = optimizer.minimize( loss,
                                    global_step=tf.train.get_global_step() )
    
    
    estimator = tf.estimator.EstimatorSpec( mode,
                                            loss=loss,
                                            train_op=train_op )
    
    return estimator

### Training Loop

In [110]:
# Define dictionary to hold the relevant parameters for the autoencoder definition function.
params = {
   "learning_rate": learning_rate,
   "latent_dimension": latent_dimension
}
    
# Get AE class    
ae_obj = get_autoencoder( full_model_def_func, # Function defining your model
                          params, # User-defined parameters. Can be any dictionary
                          model_dir, # Directory where that model will be saved.
                          force_training ) # If False, model will not train on an existing
                                           # checkpoint. Set to True to continue training
                                           # an existing model
    
# Get training and validation data. The training_input_fn will generate on infinite
# stream of synthetic data for the model to train on.
## DO NOT TOUCH
(training_input_fn,
 eval_input_fn,
 validation_images,
 validation_labels_argmax,
 classifier_params ) = get_train_and_eval_input_functions( model_dir,
                                                           False, #make_feature_detector,
                                                           None, #detector_feature_class,
                                                           False, #use_transfer_learning,
                                                           batch_size,
                                                           512, #DEFAULT_VALIDATION_SET_SIZE,
                                                           1, #validation_seed,
                                                           None, #training_seed,
                                                           0, #dali_device_id, default value
                                                           2, #dali_prefetch_batches, default value
                                                           False ) #use_mlpp_preprocessing
    
full_ae = ae_obj.full_network()
    
# Training loop
training_steps_run = 0
  
start_time = time.time()
  
while training_steps_run < total_training_steps:
        
    # Determine if we can do an eval.
    train_steps_remaining = total_training_steps - training_steps_run
    do_eval               = train_steps_remaining >= steps_per_validation
        
    # Determine how many steps we need to train this time.
    if( do_eval ):
        current_train_steps = steps_per_validation
    else:
        current_train_steps = train_steps_remaining
        
    training_steps_run += current_train_steps
        
    full_ae.train( input_fn=training_input_fn,
                   steps=current_train_steps )
       
    if( do_eval ):
            
        # Do eval.
        eval_result = full_ae.evaluate( input_fn=eval_input_fn,
                                        steps=1 )
    
        # Each step save a montage for the first 100 validation images
        predict_on_data( ae_obj,
                         validation_images[:100, :],
                         synthetic_data_params.IMAGE_SHAPE,
                         model_dir,
                         labels=validation_labels_argmax[:100],
                         image_min=synthetic_data_params.IMAGE_MIN,
                         image_max=synthetic_data_params.IMAGE_MAX,
                         delta_image_min=synthetic_data_params.DELTA_IMAGE_MIN,
                         delta_image_max=synthetic_data_params.DELTA_IMAGE_MAX,
                         data_description_string="validation" )

        # Save model
        save_model( model_dir, full_ae )
            
# Get the training time.
final_training_time = time.time() - start_time
print( "Train Time: {}".format( final_training_time ) )

INFO:tensorflow:Using config: {'_model_dir': '/domino/datasets/daniel_adams/oct_hackathon_wc/scratch/vae-bi-gru-14/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 1200, '_session_config': gpu_options {
  allow_growth: true
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2eeae1cb38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


4130 autoencoder 18:29:28 WARNING: model directory is not empty, continuing from a previous run due to FORCE_TRAINING being True: /domino/datasets/daniel_adams/oct_hackathon_wc/scratch/vae-bi-gru-14/


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /domino/datasets/daniel_adams/oct_hackathon_wc/scratch/vae-bi-gru-14/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /domino/datasets/daniel_adams/oct_hackathon_wc/scratch/vae-bi-gru-14/model.ckpt.


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: Allocating 8589934592 bytes exceeds the memory limit of 4294967296 bytes.
	 [[{{node cluster_1085_1/xla_compile}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[cluster_1083_1/merge_oidx_0/_173]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: Allocating 8589934592 bytes exceeds the memory limit of 4294967296 bytes.
	 [[{{node cluster_1085_1/xla_compile}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored.

### Make Predictions and Score Results

In [60]:
score, score_time = score_model( model_dir )

INFO:tensorflow:Restoring parameters from /domino/datasets/daniel_adams/oct_hackathon_wc/scratch/vae-gru-13/latest_model/variables/variables
Validation score for model /domino/datasets/daniel_adams/oct_hackathon_wc/scratch/vae-gru-13/ is 1.0364092588424683
Validation took 0.780996561050415 seconds


In [157]:
# This is an example function for how to load an existing model and use it to predict on data.
def post_processing( model_dir, 
                     validation_images ):

    # Load the model checkpoint
    predict_fn = tf.contrib.predictor.from_saved_model( model_dir + "/latest_model")
    
    # Predict on the images
    prediction_dict = predict_fn( {"images": validation_images} )
    
    # Look at the results
    print( prediction_dict )

In [158]:
post_processing(model_dir, validation_images)


INFO:tensorflow:Restoring parameters from /domino/datasets/daniel_adams/oct_hackathon_wc/scratch/vae-lstm-6//latest_model/variables/variables
{'rmse': array([1.0943036 , 1.025197  , 1.0825996 , 0.9256538 , 1.0511086 ,
       1.1801039 , 1.0847465 , 1.1178546 , 0.89692014, 1.0375382 ,
       0.9273722 , 1.0944893 , 0.96832776, 0.96083087, 1.0204191 ,
       1.3616666 , 0.995985  , 1.0452601 , 0.99799305, 0.8742171 ,
       1.0550975 , 0.86653996, 1.1654879 , 1.0191513 , 1.2582949 ,
       0.96122855, 1.203408  , 1.070215  , 1.0757774 , 1.0080407 ,
       1.4533216 , 0.96032965, 1.0687022 , 0.9899083 , 1.0675678 ,
       1.2646142 , 0.929964  , 1.1120285 , 1.0158714 , 1.0043011 ,
       1.2209396 , 1.1801454 , 1.034436  , 0.9959754 , 0.9674092 ,
       0.9856183 , 1.3288122 , 1.118889  , 0.99115   , 0.95166904,
       0.9698147 , 1.1066239 , 1.0129445 , 1.0454241 , 1.2125835 ,
       1.1491838 , 1.0653809 , 1.0425754 , 1.036703  , 0.98953915,
       0.92558306, 1.0524768 , 1.115516  , 0.